<a href="https://colab.research.google.com/github/Antana-A/Mini-GPT-for-Jobs-Application/blob/main/CareerGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install gradio cohere requests PyPDF2 python-docx

import gradio as gr
import cohere
import os
import requests
import shutil
from PyPDF2 import PdfReader
from docx import Document

# Set up directories for uploads
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Cohere API Key (Set it in your environment variables)
os.environ["COHERE_API_KEY"] = "sUUUiW3jwUrFokR9YZ6qNkm2XVAoUlnRyWDayTeu"  # Replace with your Cohere API key

# Extract text from PDF or DOCX
def extract_text(file_path):
    if file_path.endswith(".pdf"):
        reader = PdfReader(file_path)
        text = "".join(page.extract_text() for page in reader.pages)
    elif file_path.endswith(".docx"):
        doc = Document(file_path)
        text = "\n".join([p.text for p in doc.paragraphs])
    else:
        text = "Unsupported file format."
    return text

# Analyze resume with Cohere
def analyze_with_cohere(resume_text):
    co = cohere.Client(os.getenv("COHERE_API_KEY"))
    response = co.generate(
        model="command-xlarge-nightly",
        prompt=f"""
        Analyze this resume:
        {resume_text}

        Provide the following:
        1. Top Skills
        2. Certifications (if listed)
        3. Summary of Work Experience
        """,
        max_tokens=300,
        temperature=0.7
    )
    return response.generations[0].text.strip()

# Search jobs with Adzuna
def search_jobs_with_adzuna(job_title):
    url = "https://api.adzuna.com/v1/api/jobs/us/search/1"
    params = {
        "app_id": "2b8eddf0",  # Replace with Adzuna App ID
        "app_key": "e4d16a209133c6f92c5587c1414ed3b7",  # Replace with Adzuna App Key
        "what": job_title,
        "where": "Grand Rapids, MI",
        "distance": 10,
        "results_per_page": 5
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        jobs = response.json().get("results", [])
        return [
            {
                "title": job["title"],
                "company": job.get("company", {}).get("display_name", "N/A"),
                "location": job["location"]["display_name"],
                "description": job["description"],
                "redirect_url": job["redirect_url"]
            }
            for job in jobs
        ]
    return []

# Recommend certifications based on skills
def recommend_certifications(skills):
    co = cohere.Client(os.getenv("COHERE_API_KEY"))
    response = co.generate(
        model="command-xlarge-nightly",
        prompt=f"""
        Based on these skills: {skills}, suggest certifications to enhance career opportunities in Data Engineering, Data Science, Machine Learning, and Bioinformatics.
        """,
        max_tokens=150,
        temperature=0.7
    )
    return response.generations[0].text.strip()

# Main workflow
def ai_mini_gpt_workflow(resume):
    try:
        # Save the uploaded resume
        file_path = os.path.join(UPLOAD_FOLDER, resume.name)
        shutil.move(resume.name, file_path)

        # Extract text from the resume
        extracted_text = extract_text(file_path)

        # Analyze the resume with Cohere
        insights = analyze_with_cohere(extracted_text)
        skills = insights.split("Top Skills:")[1].split("\n")[0]  # Simple parsing

        # Fetch jobs
        roles = ["Data Engineering", "Data Science", "Machine Learning", "Bioinformatics"]
        job_results = {role: search_jobs_with_adzuna(role) for role in roles}

        # Recommend certifications
        certifications = recommend_certifications(skills)

        # Return all results
        return {
            "Resume Insights": insights,
            "Recommended Jobs": job_results,
            "Certifications": certifications
        }

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
mini_gpt_interface = gr.Interface(
    fn=ai_mini_gpt_workflow,
    inputs=gr.File(label="Upload Your Resume (PDF or DOCX)"),
    outputs=gr.JSON(label="AI Agent Output"),
    title="Mini GPT for Jobs and Certifications",
    description="Upload your resume to analyze skills, certifications, and find job recommendations in Grand Rapids, MI."
)

# Launch Gradio app
mini_gpt_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://430acdc601f69c4dfb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import gradio as gr
import os
import json

# File to store feedback
FEEDBACK_FILE = "./feedback.json"

# Feedback storage function
def collect_feedback_with_followup(results, feedback_action):
    try:
        # Load existing feedback
        if os.path.exists(FEEDBACK_FILE):
            with open(FEEDBACK_FILE, "r") as f:
                feedback_data = json.load(f)
        else:
            feedback_data = []

        # Add new feedback
        feedback_data.append({
            "results": results,
            "feedback": feedback_action  # 'thumbs_up' or 'thumbs_down'
        })

        # Save feedback
        with open(FEEDBACK_FILE, "w") as f:
            json.dump(feedback_data, f, indent=4)

        # Provide follow-up suggestion
        if feedback_action == "thumbs_up":
            return "Thank you for your feedback! Let me know if you’d like another search or more suggestions!"
        elif feedback_action == "thumbs_down":
            return "Sorry to hear that! Would you like to refine your search or try again with a new query?"
        else:
            return "Feedback received. Let me know if you’d like further refinements or additional features!"

    except Exception as e:
        return f"An error occurred while saving feedback: {str(e)}"

# Gradio interface with follow-up workflow
with gr.Blocks() as feedback_interface:
    with gr.Row():
        gr.Markdown("## Mini GPT for Jobs and Certifications with Feedback")

    # Step 1: Upload Resume
    with gr.Row():
        resume_input = gr.File(label="Upload Your Resume (PDF or DOCX)")
        analyze_button = gr.Button("Analyze Resume")

    # Step 2: Display Results
    with gr.Row():
        results_output = gr.JSON(label="AI Agent Output")

    # Step 3: Feedback Buttons and Follow-Up
    with gr.Row():
        feedback_output = gr.Textbox(label="Feedback Response", interactive=False)
        thumbs_up_button = gr.Button("👍")
        thumbs_down_button = gr.Button("👎")
        refine_search_button = gr.Button("Refine Search")

    # Workflow Connections
    analyze_button.click(
        fn=ai_mini_gpt_with_feedback,  # Call the analysis function
        inputs=[resume_input],
        outputs=[results_output]
    )

    thumbs_up_button.click(
        fn=collect_feedback_with_followup,  # Feedback function for thumbs up
        inputs=[results_output, gr.Textbox("thumbs_up", visible=False)],
        outputs=[feedback_output]
    )

    thumbs_down_button.click(
        fn=collect_feedback_with_followup,  # Feedback function for thumbs down
        inputs=[results_output, gr.Textbox("thumbs_down", visible=False)],
        outputs=[feedback_output]
    )

    # Option to refine the search
    refine_search_button.click(
        fn=lambda: "Let’s refine the results. Please upload your resume again or specify what you'd like improved.",
        inputs=[],
        outputs=[feedback_output]
    )

feedback_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f43eeeba5eab4a03f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
